In [4]:
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 762.7 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.1/23.1 MB 7.1 MB/s eta 0:00:0000:0100:01


In [1]:
import torchvision

In [6]:
import torch

In [40]:
class ResNetBlock(torch.nn.Module):
    def __init__(self, d):
        super().__init__()
        self.conv_blk_1 = torch.nn.Conv2d(d, d, kernel_size=(3, 3), padding=1)
        self.relu = torch.nn.ReLU()
        self.conv_blk_2 = torch.nn.Conv2d(d, d, kernel_size=(3, 3), padding=1)

    def forward(self, x):
        return self.conv_blk_2(self.relu(self.conv_blk_1(x))) + x

In [84]:
import copy

In [86]:
encoder_ = {
    "layers": [
        {"name": "conv2d", "in_channels": 3, "out_channels": 64, "kernel_size": 7, "stride": 1},
        {"name": "conv2d", "in_channels": 64, "out_channels": 64, "kernel_size": 3, "stride": 2, 'padding': 1},
        {"name": "resnet", "repeat": 2, "in_channels": 64, "out_channels": 64, "kernel_size": 3, "stride": 1, "padding": 1},
        {"name": "conv2d", "in_channels": 64, "out_channels": 128, "kernel_size": 3, "stride": 2, 'padding': 1},
        {"name": "resnet", "repeat": 2, "in_channels": 128, "out_channels": 128, "kernel_size": 3, "stride": 1, "padding": 1},
        {"name": "conv2d", "in_channels": 128, "out_channels": 256, "kernel_size": 3, "stride": 2, 'padding': 1},
        {"name": "resnet", "repeat": 2, "in_channels": 256, "out_channels": 256, "kernel_size": 3, "stride": 1, "padding": 1}
    ]
}

In [90]:
encoder = copy.deepcopy(encoder_)

In [91]:
layers = []

for layer in encoder['layers']:
    if layer['name'] == 'conv2d':
        del layer['name']
        layers.append(torch.nn.Conv2d(**layer))
        continue
    if layer['name'] == 'resnet':
        for _ in range(layer['repeat']):
            layers.append(ResNetBlock(d=layer['in_channels']))

In [68]:
x = torch.randn(2, 3, 256, 256)

In [69]:
x.shape

torch.Size([2, 3, 256, 256])

In [70]:
for l in torch.nn.ModuleList(layers):
    x = l(x)

In [71]:
x.shape

torch.Size([2, 256, 32, 32])

In [93]:
z = torch.randn(32, 256)

In [94]:
lookup = torch.nn.Embedding(1024, 256)

In [99]:
z.unsqueeze(0).repeat(1024, 1, 1) - lookup.weight.unsqueeze(1)

torch.Size([1024, 32, 256])

In [96]:
z - lookup.weight

RuntimeError: The size of tensor a (32) must match the size of tensor b (1024) at non-singleton dimension 0